In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ratings = pd.read_csv("ml-100k/u.data", sep="\t", names=["user_id", "movie_id", "rating", "timestamp"])
movies = pd.read_csv("ml-100k/u.item", sep="|", names=["movie_id", "movie_title"], usecols=[0,1], encoding="latin-1")

In [ ]:
print(ratings.head())

In [ ]:
print(movies.head())

In [5]:
df = ratings.merge(movies, on ="movie_id")

In [ ]:
print(df.head())

In [ ]:
num_ratings = len(ratings)
num_movies = ratings["movie_id"].nunique()
num_users = ratings["user_id"].nunique()

print(f"Number of ratings: {num_ratings}")
print(f"Number of movies: {num_movies}")
print(f"Number of users: {num_users}")
print(f"The number of elements within the user-movie matrix is: {num_users * num_movies} elements")
print(f"The sparsity of the user_movie matrix is: {(num_ratings / (num_users * num_movies)) * 100:.2f}%")
print(f"We can conclude from {(num_ratings / (num_users * num_movies)) * 100:.2f}% that the user-movie matrix is very sparse.")

In [ ]:
sns.countplot(x="rating", data=ratings)
plt.title("Distribution of movie ratings", fontsize=14)
plt.show()

In [ ]:
print(f"Mean global rating: {round(ratings['rating'].mean(), 2)}")

In [ ]:
mean_ratings = ratings.groupby("user_id")["rating"].mean()
print(f"Mean rating per user: {round(mean_ratings.mean(), 2)}")